In [67]:
# Standard library imports
import os
import re
import csv
import sys
import html
from datetime import datetime
from collections import defaultdict

# Third-party library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
import psutil
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Local application/library specific imports
import utils

### Data from scratch - Pandas

In [13]:
#How is my data delimited?
path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_bios_2023-05-19.csv'

def print_lines(path, file, start_line=0, end_line= 5):
    with open(f"{path}/{file}", 'r') as f:
        for i in range(end_line):
            line = f.readline()
            if i >= start_line:
                print(line)

print_lines(path, file)


#The data is comma delimited

twitter_id,id,screen_name,name,description,url,timestamp_utc,local_time,location,verified,protected,tweets,followers,friends,likes,lists,image,default_profile,default_profile_image,witheld_in_countries,witheld_scope

3342215494,3342215494,titisanogo8,Titi sanogo,Je crois en DIEU et à mon travail j'y arriverai.....,,1435017944,2015-06-23T00:05:44,"Ile-de-France, France",0,0,6,44,733,91,0,https://pbs.twimg.com/profile_images/1249394390029742081/xuVolLn6_normal.jpg,1,0,,

3115495713,3115495713,AndreDeybach,DEYBACH André,,,1427309108,2015-03-25T18:45:08,,0,0,0,1,40,0,0,https://pbs.twimg.com/profile_images/580803690757533697/pHNcCBLh_normal.jpg,1,0,,

244075010,244075010,matttownley1985,Matt Townley,"Hotelier, traveller, fan of all things hospitality, great food and fine wine! All views my own etc!!",,1296220595,2011-01-28T13:16:35,"Manchester, England",0,1,2535,772,1264,1251,7,https://pbs.twimg.com/profile_images/928075998930681856/ZFXboKc3_normal.jpg,0,0,,

2986463442,2986463442,alex_guev

In [56]:
# # How much memory available?
# def get_available_memory():
#     return psutil.virtual_memory().available

# available_memory = get_available_memory()
# print(f"Available memory: {available_memory / (1024 * 1024 * 1024)} GB")


# # check size of loaded df and print 
# def get_df_memory_usage(df):
#     return df.info(memory_usage='deep')

# get_df_memory_usage(df)

## 1. Filtering french brands

Here, we'll aim to select only the french brands. 
- One approach is to use language detection. Via the gcd3 library (?)
- Another approach could be to look at location


Plan:
- Load the df of the brands and their bios. 
- Filter
- Now, filter the reduced follower df based on these final french brands

- are all users now french ? (look at combination of language and location - potentially drop those that does not have any usable indicator)


- Might need to manually inspect these later on

In [66]:


#lod the markers_bios_2023-05-19.csv file
path = '/home/livtollanes/SocialMarkers'
file = 'markers_bios_2023-05-19.csv'

req_cols = ['twitter_name', 'id', 'screen_name', 'description', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

dtypes = {'twitter_name':'object', 
          'id': 'float64',
          'screen_name': 'object', 
          'description': 'object',
          'location': 'object',
          'tweets': 'float64',
          'followers': 'float64',
          'friends': 'float64',
          'likes': 'float64',
          'lists': 'float64',
          'timestamp_utc': 'float64'}

dfm = file_

TypeError: load_file() takes 3 positional arguments but 4 were given

In [ ]:
# Remove emojis, weird font, and detect language in descriptions

from unidecode import unidecode

def remove_emoji(string):
    return emoji.demojize(string, delimiters=("", ""))

def convert_to_regular_script(string):
    return unidecode(string)

def detect_language(bio):
    try:
        return detect(bio)
    except LangDetectException:
        return 'unknown'

dfm['description_noems'] = dfm['description'].apply(lambda bio: remove_emoji(bio) if pd.notnull(bio) else '')
dfm['description_noems'] = dfm['description_noems'].apply(lambda bio: convert_to_regular_script(bio) if pd.notnull(bio) else '')
dfm['language'] = dfm['description_noems'].apply(lambda bio: detect_language(bio) if bio.strip() != '' else 'unknown')

In [ ]:
dfm_french = dfm[dfm['language'] == 'fr']

#now create an "other" df with all the other languages
dfm_other = dfm[dfm['language'] != 'fr']

In [ ]:
#print how many rows are in the dfm_french df and add fstring
print(f"Number of rows in dfm_french: {dfm_french.shape[0]}")

#print how many rows are in the dfm_other df and add fstring
print(f"Number of rows in dfm_other: {dfm_other.shape[0]}")

In [ ]:
## Manually adding faulty identified fr languages to the frencch df
# Select the rows
#selected_rows = dfm_other.loc[[1, 21, 28, 30, 36, 39, 41, 48, 78, 92, 94, 96, 101, 106, 119, 121, 124, 128, 129, 131, 133, 135, 181, 182, 199, 215, 230, 231, 233, 236,
#226, 227, 221,208, 202, 178, 180, 171, 113, 38, 37]]

# Print the selected rows for inspection
pd.set_option('display.max_colwidth', None)
#print(selected_rows[['twitter_name','description_noems']])

# If the selected rows are correct, merge them into dfm_french and remove them from dfm_other
dfm_french = pd.concat([dfm_french, selected_rows])
dfm_other = dfm_other.drop([1, 21, 28, 30, 36, 39, 41, 48, 78, 92, 94, 96, 101, 106, 119, 121, 124, 128, 129, 131, 133, 135, 181, 182, 199, 215, 230, 231, 233, 236, 226, 227, 221,208, 202, 178, 180, 171, 113, 38, 37])

In [ ]:
pd.set_option('display.max_rows', None)
print(dfm_french[['twitter_name']])

## 2. Filtering out irrelevant users 

- In this part, we want to remove users that don't follow enough brands. 
- This is to match the procedure of He and Tsvetkova (2023)
- The purpose is to ensure we have enough information to generate SES estimates for users.

In [82]:
#Take a look at the data (Of the markers and their followers)
# path = '/home/livtollanes/SocialMarkers'
# file = 'markers_followers_2023-05-19.csv'


# def print_lines(path, file, start_line=0, end_line=10):
#     with open(f"{path}/{file}", 'r') as f:
#         for i in range(end_line):
#             line = f.readline()
#             if i >= start_line:
#                 print(line)

# print_lines(path, file)


id,cursor,follower_id

415859364,,1655336804831174657

415859364,,1659648141497454593

415859364,,1525534139478310915

415859364,,1659648883209674764

415859364,,1659648836594458626

415859364,,881616301

415859364,,1618696506424303637

415859364,,1659647872202055692

415859364,,1659647973637357568



#### Identify the number of brands each unique follower follows

Create dictionary of [keys: follower_Id, Value: Brand_id]
Without loading the data into python

In [100]:
# Create a dictionary of keys:follower_id and value: brands
brands_per_follower = defaultdict(set)

# Open the CSV file
with open('/home/livtollanes/SocialMarkers/markers_followers_2023-05-19.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        # Add the brand id to the set of brands for this follower
        brands_per_follower[row['follower_id']].add(row['id'])


# Convert the sets to counts to see how many brands each follower follows
brands_per_follower_count = {follower_id: len(brands) for follower_id, brands in brands_per_follower.items()}

In [101]:
#How many unique values (brands) are there in the dictionary?

unique_values = set(value for values in brands_per_follower.values() for value in values)

# The number of unique values is the length of the set
num_unique_values = len(unique_values)

print(f"The number of unique values in the dictionary is {num_unique_values}.")

The number of unique values in the dictionary is 236.


In [102]:
# How many keys (follower_id) are there in the dictionary?
num_keys = len(brands_per_follower_count)

print(f"The number of keys in the dictionary is {num_keys}.")

The number of keys in the dictionary is 70636295.


In [103]:
#inspect first key value pairs 
# Get an iterator over the dictionary's items
items = iter(brands_per_follower_count.items())

# Get the first 5 items
for _ in range(5):
    print(next(items))  

('1655336804831174657', 4)
('1659648141497454593', 3)
('1525534139478310915', 3)
('1659648883209674764', 1)
('1659648836594458626', 1)


In [104]:
# How many users follow less than num_brands brands?
num_brands = 5

# How many users follow less than num_brands brands?
# Assuming brands_per_follower_count is your dictionary
count = sum(1 for value in brands_per_follower_count.values() if value < num_brands)

# Calculate the percentage
percentage = (count / len(brands_per_follower_count)) * 100

# Calculate the percentage of the remaining data
remaining_percentage = 100 - percentage

# Calculate the number of remaining users
remaining_users = len(brands_per_follower_count) - count

print(f"The number of keys that follow less than {num_brands} IDs is {count}, which is {percentage:.2f}% of the total. Removing these leaves {remaining_percentage:.2f}% of the data, or {remaining_users} users.")

The number of keys that follow less than 5 IDs is 66693204, which is 94.42% of the total. Removing these leaves 5.58% of the data, or 3943091 users.


In [105]:
# excluding users that follow less than 5 brands in our full dict

num_brands = 5

# Filter the dictionary
filtered_brands_per_follower = {follower_id: brands for follower_id, brands in brands_per_follower.items() if len(brands) >= num_brands}

#inspect first key value pairs in filtered dict
items = iter(filtered_brands_per_follower.items())

# Get the first 5 items
for _ in range(5):
    print(next(items))

('1543191014579617793', {'415859364', '326359913', '88944305', '348379865', '38164846'})
('1257705208592756737', {'60659498', '326359913', '44084633', '348379865', '60054156', '415859364'})
('1301293067945807877', {'554957168', '88944305', '21915474', '44084633', '415859364'})
('1644463375231885315', {'60659498', '88944305', '16096416', '44084633', '348379865', '60054156', '415859364'})
('1233001848006729729', {'40448264', '83864876', '112754792', '33893706', '19976004', '88944305', '47902100', '133663801', '415859364'})


In [ ]:
# Save the list of users that followe more than five brands to a .pkl file
with open('/home/livtollanes/NewData/followfive_filtered.pkl', 'wb') as f:
    pickle.dump(filtered_brands_per_follower, f)


#load the pickle list from file
# with open('/path/to/your/directory/followers_filtered.pkl', 'rb') as f:
#     followers_filtered = pickle.load(f)

Now, filter the follower df

In [58]:

#Loading in one data set
path = '/home/livtollanes/SocialMarkers'
file = 'markers_followers_bios_2023-05-19.csv'




req_cols = ['twitter_id', 'id', 'screen_name', 'description', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

dtypes = {
    'twitter_id': 'int64',
    'id': 'float64',
    'screen_name': 'object',
    'description': 'object',
    'location': 'object',
    'tweets': 'float64',
    'followers': 'float64',
    'friends': 'float64',
    'witheld_in_countries': 'float64'
}

df = load_file(path, file, req_cols)



In [46]:
# Load the dictionary from the .pkl file
with open('/home/livtollanes/NewData/followfive_filtered.pkl', 'rb') as f:
    followers_dict = pickle.load(f)
    
#make a list of the keys in the dictionary
followers = list(followers_dict.keys())


In [62]:
# Convert the dictionary items to a list
items = list(followers_dict.items())

# Get the first few entries
first_entries = items[:5]

# Print the first few entries
for key, value in first_entries:
    print(f"{key}: {value}")

1543191014579617793: {'415859364', '326359913', '88944305', '348379865', '38164846'}
1257705208592756737: {'60659498', '326359913', '44084633', '348379865', '60054156', '415859364'}
1301293067945807877: {'554957168', '88944305', '21915474', '44084633', '415859364'}
1644463375231885315: {'60659498', '88944305', '16096416', '44084633', '348379865', '60054156', '415859364'}
1233001848006729729: {'40448264', '83864876', '112754792', '33893706', '19976004', '88944305', '47902100', '133663801', '415859364'}


In [63]:
# Search for 1233001848006729729 in 'twitter_id' column
mask = df['twitter_id'] == 1233001848006729729

# Use the mask to select the rows where 'twitter_id' is 1233001848006729729
matching_rows = df[mask]

# Print the matching rows
print(matching_rows)

                   twitter_id            id      screen_name  \
44884750  1233001848006729729  1.233002e+18  OmarKHA95647383   

                                description  timestamp_utc location  tweets  \
44884750  ‏‏كل أرض يرفع فيها الآذان هي وطني   1.582806e+09      NaN   248.0   

          followers  friends   likes  lists  
44884750      193.0   3862.0  4242.0    0.0  


In [69]:
# Convert the keys in followers_dict to integers
follower_ids = [int(key) for key in followers_dict.keys()]

# Filter the DataFrame to only include the rows where 'twitter_id' is in the follower_ids list
filtered_df = df[df['twitter_id'].isin(follower_ids)]

# Print the first few rows of the filtered DataFrame
print(filtered_df.head())

            twitter_id            id    screen_name  \
0           3342215494  3.342215e+09    titisanogo8   
1           3115495713  3.115496e+09   AndreDeybach   
7   738440598865088512  7.384406e+17    isusername1   
28           912721573  9.127216e+08     Armelrichy   
35            30797693  3.079769e+07  AVMGDIGITALHD   

                                          description  timestamp_utc  \
0   Je crois en DIEU et à mon travail j'y arrivera...   1.435018e+09   
1                                                 NaN   1.427309e+09   
7                                                 NaN   1.464893e+09   
28                                                NaN   1.351526e+09   
35  THE NEW DIGITAL STATION!!! \nfollowed by @ROCN...   1.239593e+09   

                 location   tweets  followers  friends  likes  lists  
0   Ile-de-France, France      6.0       44.0    733.0   91.0    0.0  
1                     NaN      0.0        1.0     40.0    0.0    0.0  
7                     N

In [74]:
# Print the number of rows in the DataFrame
print(f"Number of rows: {filtered_df.shape[0]}")
print(f"Number of columns: {filtered_df.shape[1]}")

Number of rows: 3943091
Number of columns: 11


Remove all users that have sent less than 100 tweets

In [67]:
#check the dtype ow tweets in filtered df
print(filtered_df['tweets'].dtype)

float64


In [73]:
# Filter the DataFrame to only include users with more than 100 tweets
filtered_df2 = filtered_df[filtered_df['tweets'] > 99]

#print the number of rows in the filtered df
print(f"Number of rows after removing all with less than 100 tweets: {filtered_df2.shape[0]}")

#Print how many rows were removed
print(f"Number of rows removed: {filtered_df.shape[0] - filtered_df2.shape[0]}")

Number of rows after removing all with less than 100 tweets: 1381356
Number of rows removed: 2561735


In [75]:
#  Only include users with more than 25 followers
filtered_df3 = filtered_df2[filtered_df2['followers'] > 24]

# Print the number of rows in the filtered DataFrame   
print(f"Number of rows after removing all with less than 25 followers: {filtered_df3.shape[0]}")

# Print how many rows were removed
print(f"Number of rows removed: {filtered_df2.shape[0] - filtered_df3.shape[0]}")

Number of rows after removing all with less than 25 followers: 1192532
Number of rows removed: 188824


In [79]:
#Find the most recent timestamp
most_recent_timestamp = filtered_df3['timestamp_utc'].max()
print(f"Most recent timestamp: {most_recent_timestamp}")

#convert the timestamp to a datetime object
most_recent_datetime = datetime.fromtimestamp(most_recent_timestamp)
print(f"Most recent datetime: {most_recent_datetime}")


#Do the same for the oldest timestamp
oldest_timestamp = filtered_df3['timestamp_utc'].min()
print(f"Oldest timestamp: {oldest_timestamp}")

#convert the timestamp to a datetime object
oldest_datetime = datetime.fromtimestamp(oldest_timestamp)
print(f"Oldest datetime: {oldest_datetime}")


Most recent timestamp: 1684644667.0
Most recent datetime: 2023-05-21 04:51:07
Oldest timestamp: 1153057347.0
Oldest datetime: 2006-07-16 13:42:27


In [81]:
# Now write the filtered_df3 to a csv file
# filtered_df3.to_csv('/home/livtollanes/NewData/filtered_follower_bios_.csv', index=False)

### Filterings
- Follow at least five brands
- sent at least 100 tweets
- have at least 25 followers
- sent at least five tweets in the first few months of the year the data was collected (maybe not relevant for this data - the twitter bios data does not contain the tweets. Only creation date for the profile)